In [1]:
#!/usr/bin/env python3

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
def bin_data(reco_data, sim_data, bins, quant=68, logspace=True):
    """
    reco_data: energy distribution of reconstructed events or after cut. as np.array
    reco_data: energy distribution of reconstructed events or after cut. as np.array
    
    return: return the bin centers, and ratio of passing (triggers, reco, cut) to simulated.
    bins : Number of bins. Shoud be an interger not a list.
    """
    if logspace:
        bins = np.logspace(np.log10(np.min(reco_data)), np.log10(np.max(reco_data)), bins)
    else:
        bins = np.linspace(np.min(reco_data), np.max(reco_data), bins)

    hist1, bin_edges1 = np.histogram(reco_data, bins=bins)
    hist2, bin_edges2 = np.histogram(sim_data, bins=bins)
    ratio = np.divide(hist1, hist2, where=(hist2 !=0))
    
    print(f'# reco event: {hist1} events')
    print(f'# sim events: {hist2} events')
    print(f'# efficiency: {np.round(ratio, 3)} events')
    
    centers = (bins[1:] + bins[:-1]) / 2.0

    return (
        np.array(centers),
        np.array(ratio)
    )

In [ ]:
# ----------------------------------------------------------------------

df = pd.read_csv('likelihood_mmsreco_16pmts_mc_truth_seed_70str_standard_unclean_selection.csv', index_col=False)

df2 = pd.read_csv('simulation_sim0005_triggered_16pmts_mc_truth_seed_70str_standard_unclean_selection.csv')


# <span style='color:Black'> Data preprocessing </span>

In [3]:
# Checking NaN on entire DataFrame
print(df.isnull().values.any())
# Counte NaN on entire DataFrame
#df = df.dropna(subset=['dirTrackLengthA_reco'])
print(df.isnull().sum())

print('\n')

# Checking NaN on entire DataFrame
print(df2.isnull().values.any())
# Counte NaN on entire DataFrame
print(df2.isnull().sum())

bins = 11
energy_min = 1e3
energy_max = 1e6

In [ ]:
df = df.loc[(df['muon_energy'] >= energy_min)]
df2 = df2.loc[(df2['muon_energy'] >= energy_min)]
# Make reference using LDirA > 0 as denominator
#df2 = df.loc[(df['dirTrackLengthA_reco'] > 0)]

In [5]:
# ----------------------------------------------------------------------

plt.figure(dpi=300)
# Efficiency vs muon energy
for LDir in [0, 100, 200, 400, 700]:
    data = df.loc[(df['dirTrackLengthA_reco'] > LDir)]
    reco_energy = data.muon_energy
    sim_energy = df2.muon_energy
    bin_centers, efficiency = bin_data(
        reco_energy, sim_energy, bins=bins, logspace=True
    )
    print('\n\n')
    plt.plot(bin_centers, efficiency, "-", label=f'LDirA > {LDir} m')
plt.xlabel("Muon energy [GeV]")
plt.ylabel("Efficiency")
plt.ylim(0, 1.0)
plt.xlim(energy_min, energy_max)
plt.xscale("log")
plt.grid(b=None, which="both", axis="both", linestyle="--", linewidth=0.3)
plt.legend()
plt.savefig("mmsreco_angular_error_efficiency.png")
plt.show()

,event_id,angular_error_linefit,angular_error_mmsreco,angular_error_splines_35ns,angular_error_splines_20ns,angular_error_splines_10ns,angular_error_splines_05ns,logl_splines_35ns,logl_splines_20ns,logl_splines_10ns,logl_splines_05ns,logl_mmsreco,nchannels_count,qtotal_clean,qtotal_unclean,nhits_clean,nhits_unclean,zenith_angle,muon_energy,dirTrackLengthA_reco
0,4.0,0.487677,0.109332,0.598010,0.270942,0.217696,0.006116,2879.779777,2745.899079,2598.279414,2480.509945,2343.115684,63.0,2844.136218,3201.873678,517.0,869.0,150.662855,512903.553092,915.748829
1,9.0,116.275724,83.227653,81.310991,83.161634,83.135916,83.297585,248.280952,247.550087,247.290345,247.158064,247.005134,11.0,79.610525,468.841221,41.0,424.0,105.507968,11354.850029,NaN
2,12.0,0.181515,0.170790,0.242992,0.232130,0.122696,0.181000,678.998556,649.016390,616.140479,591.549206,568.496264,36.0,335.144792,700.630904,118.0,474.0,142.081745,17764.540622,968.139153
3,13.0,4.734283,0.172002,2.577853,1.582213,0.842176,0.128831,108.864398,99.335567,91.637313,85.907228,72.414210,8.0,40.449310,366.870945,22.0,345.0,31.642455,1786.377278,465.954543
4,14.0,2.429265,3.315244,4.769528,3.444290,3.240858,3.055932,252.508423,232.102993,213.014226,201.210634,189.346561,11.0,786.244719,1067.619258,48.0,324.0,46.952518,19814.882604,200.199007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293184,489.0,155.705780,120.540935,119.336943,120.297201,120.413764,120.550193,165.083890,165.035099,164.974066,164.933075,164.908313,8.0,27.941514,382.254641,17.0,374.0,71.963934,52549.088219,NaN
293185,490.0,1.708691,0.017255,1.086974,0.102729,0.076083,0.058836,1939.751888,1823.935646,1707.745229,1609.186348,1493.774424,47.0,2787.833402,3132.304609,351.0,695.0,92.996455,507285.577759,879.021947
293186,492.0,21.252562,26.426626,25.527996,26.412424,26.430826,26.428832,396.409230,392.136229,392.012270,391.787484,391.579061,19.0,118.212451,459.260647,75.0,406.0,90.655452,186959.461117,NaN
293187,493.0,90.433929,NaN,90.433929,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,10.808382,368.199126,9.0,373.0,31.042847,2823.378348,NaN


In [ ]:
# ----------------------------------------------------------------------

bins = 11
bins = np.logspace(np.log10(np.min(df2.muon_energy)), 
                   np.log10(np.max(df2.muon_energy)), bins)
plt.figure(dpi=300)
plt.hist(df2.muon_energy, bins=bins)
plt.xlabel("Muon energy [GeV]")
plt.ylabel("Counts")
plt.xscale('log')
plt.grid(b=None, which="both", axis="both", linestyle="--", linewidth=0.3)
plt.savefig("mmsreco_event_energy_distribution.png")
plt.show()